<a href="https://colab.research.google.com/github/Hamza-Ali0237/PyTorch-Projects/blob/main/Intermediate/PyTorch-FineGrained-TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Grained Transfer Learning

In [ ]:
!pip install torchmetrics

In [ ]:
import torch
from torch import nn
import torchvision as tv
import torchmetrics as tm
from torchvision import transforms
from torchvision.datasets import Flowers102
from torch.utils.data import Dataset, DataLoader

import random
import numpy as np
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

# Configuration

In [ ]:
conf_dict = {
    # Reproducibility
    "seed": 42,

    # Computing Device
    "device": "cuda" if torch.cuda.is_available() else "cpu",

    # Data Settings
    "root_dir": "./data",
    "image_size": 224,
    "batch_size": 32,
    "num_workers": 2,
    "pin_memory": True,

    # Model Settings
    "architecture": "resnet50",   # Backbone Architecture
    "pretrained": True,
    "num_classes": 102,

    # Training Hyperparameters
    "epochs": 25,
    "lr_head": 1e-3,              # LR for classifier head
    "lr_backbone": 1e-4,          # Lower LR for fine-tuning backbone
    "weight_decay": 1e-4,
    "label_smoothing": 0.0,

    # Scheduler
    "scheduler": "onecycle",

    # Imbalance Handling
    "use_class_weights": True,
    "use_weighted_sampler": False,

    # Logging & Saving
    "log_every_n_steps": 20,
    "save_best_by": "val_loss",
    "checkpoint_dir": "No checkpoints for now"
}

print(conf_dict)

In [ ]:
device = conf_dict["device"]

In [ ]:
# Set seed for reproducibility
SEED = conf_dict["seed"]

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
  torch.cuda.manual_seed(SEED)
  torch.cuda.manual_seed_all(SEED)

# Ensure deterministic behavior
## for speed over strict determinism, set deterministic=False and benchmark=True
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Data Transforms

In [ ]:
# Normalization values (ImageNet)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Traning Transforms (Augmentaion + Normalization)
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(
        brightness = 0.2, contrast = 0.2, saturation = 0.2, hue = 0.1
    ),
    transforms.ToTensor(),
    transforms.Normalize(
        mean = mean, std = std
    )
])

# Validation & Test Transforms (No heavy augmentation, just resize and center)
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean = mean, std = std
    )
])

test_transforms = val_transforms

# Load Data

In [ ]:
# Load Datasets
train_dataset = Flowers102(root=conf_dict["root_dir"], split="train", transform=train_transforms, download=True)
val_dataset = Flowers102(root=conf_dict["root_dir"], split="val", transform=val_transforms, download=True)
test_dataset = Flowers102(root=conf_dict["root_dir"], split="test", transform=test_transforms, download=True)

In [ ]:
# Define Default DataLoaders
train_loader = DataLoader(
    train_dataset, batch_size = conf_dict["batch_size"], shuffle = True,
    num_workers = conf_dict["num_workers"], pin_memory = conf_dict["pin_memory"]
)

val_loader = DataLoader(
    val_dataset, batch_size = conf_dict["batch_size"], shuffle = False,
    num_workers = conf_dict["num_workers"], pin_memory = conf_dict["pin_memory"]
)

test_loader = DataLoader(
    test_dataset, batch_size = conf_dict["batch_size"], shuffle = False,
    num_workers = conf_dict["num_workers"], pin_memory = conf_dict["pin_memory"]
)

In [ ]:
# Check Class Imbalance

# Count samples per class in training set
targets = train_dataset.targets
class_counts = Counter(targets)

# Calculate weights: total_samples / (num_classes * count[class])
classes = train_dataset.classes
num_classes = len(classes)
total_samples = len(train_dataset)
class_weights = []

for c in range(num_classes):
  weight = total_samples / (num_classes * class_counts[c])
  class_weights.append(weight)

# Convert to tensor
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Define loss with weights
criterion = nn.CrossEntropyLoss(weight = class_weights)

In [ ]:
# Plot distributions (counts vs weights)

# Convert counts into a list aligned with class indices
counts_list = [class_counts[c] for c in range(num_classes)]

# Print first 10 classes as a quick sanity check
for i in range(10):
    print(f"Class {i:3d} | Count: {counts_list[i]:4d} | Weight: {class_weights[i].item():.4f}")

fig, ax1 = plt.subplots(figsize=(14,6))

# Plot counts on left y-axis
ax1.bar(range(num_classes), counts_list, alpha=0.6, label="Class Counts")
ax1.set_xlabel("Class Index")
ax1.set_ylabel("Number of Samples")
ax1.legend(loc="upper left")

# Plot weights on right y-axis
ax2 = ax1.twinx()
ax2.plot(range(num_classes), class_weights.cpu().numpy(), 'r-', label="Class Weights")
ax2.set_ylabel("Class Weight")
ax2.legend(loc="upper right")

plt.title("Class Distribution vs. Computed Weights")
plt.show()

# Model Definition

In [ ]:
# Load pretrained ResNet18
model = tv.models.resnet18(
    weights = models.ResNet18_Weights.DEFAULT
)

# Freeze all model parameters
for params in model.parameters():
  params.requires_grad = False

# Get the number of input features to the final layer
in_features = model.fc.in_features

# Replace the final fc layer to match 102 classes
model.fc = nn.Linear(in_features, 102)

# Unfreeze only classifier parameters
for params in model.fc.parameters():
  param.requires_grad = True

model.to(device)

# Double-check which params will be trained
for name, param in model.named_parameters():
    if param.requires_grad:
        print("Trainable:", name)

In [ ]:
print(model)

# Model Training

In [ ]:
# Define Optmizer and Scheduler
optimizer = torch.optim.Adam(
    model.fc.parameters(),
    lr = conf_dict["head_lr"],
    weight_decay = conf_dict["weight_decay"]
)

scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size = conf_dict["step_size"],
    gamma = conf_dict["gamma"]
)